<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/How_to_create_dynamic_circuits_with_mid_circuit_measurements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

So the application of mid-circuit measurement are so may for example, in quantum teleportation, error correction, error mitigation, and others.

Before going into these topics, it is important to familiarize onesef with the syntax and features in mid-circuit measurements(MCMs).

In this tutorials, we will work in dynamic quantum circuits that use control flow based MCMs.

***Dynamic circuit with a single MCM and conditional***

We start with a minimal dynamic circuit on two qubits.

1. It rotates one qubit about the X-axis and applies a phase to the other qubit using a T gate.

2. Both qubits are entangled with a CNOT gate.

3. The second qubit is measured with measure().

4. If we measured a 1 in the previous step, an S gate is applied to the first qubit. This conditioned operation is realized with cond().

5. Finally, the expectation value of the Pauli Y operator on the first qubit is returned.

In [19]:
 !pip install pennylane --quiet

In [20]:
import pennylane as qml
import numpy as np

dev = qml.device("lightning.qubit", wires=2)

@qml.qnode(dev, interface="numpy")
def circuit(x):
  qml.RX(x, wires=0)
  qml.T(wires=1)
  qml.CNOT(wires = [0,1])
  mcm = qml.measure(1)
  qml.cond(mcm,qml.S)(wires=0)

  return qml.expval(qml.Y(0))


x = 1.5
print(circuit(x))

0.0


***How to dynamically prepare half-filled basis states***

We now turn to a more complex example of a dynamic circuit. We will build a circuit that non-deterministically initializes half-filled computational basis states, i.e., basis states with as many 1s as 0s.

In [21]:
def init_state(x):
  #rotate the first three qubits
  for w in range(3):
    qml.RX(x[w],w)

  qml.CNOT([0,1])
  qml.CNOT([1,2])
  qml.CNOT([2,0])



With this subroutine in our hands, let’s define the full QNode. For this, we also create a shot-based device.



In [22]:
shots = 100
dev = qml.device("default.qubit", shots=shots)


@qml.qnode(dev)
def create_half_filled_state(x):
  init_state(x)
  for w in range(3):
    #measure one qubit at a time and flip another, fresh qubit if measured 0
    mcm=qml.measure(w)
    qml.cond(~mcm, qml.X)(w+3) #this applied a conditional operation, if measurement result mcm is 0, it applies an X gate to the qubit at wire 'w+3'. Essentially it flips the state of qubits 3,4 and 5 if qubits 0,1 adn 2 are measured to be in state 0

  return qml.counts(wires=range(6))


Before running this QNode, let's sample some random input parameters and draw the circuit

In [23]:
np.random.seed(652)

x=np.random.random(3)*np.pi

print(qml.draw(create_half_filled_state)(x))


0: ──RX(1.33)─╭●────╭X──┤↗├────────────────────┤ ╭Counts
1: ──RX(1.63)─╰X─╭●─│────║──────┤↗├────────────┤ ├Counts
2: ──RX(0.92)────╰X─╰●───║───────║──────┤↗├────┤ ├Counts
3: ──────────────────────║───X───║───────║─────┤ ├Counts
4: ──────────────────────║───║───║───X───║─────┤ ├Counts
5: ──────────────────────║───║───║───║───║───X─┤ ╰Counts
                         ╚═══╝   ║   ║   ║   ║          
                                 ╚═══╝   ║   ║          
                                         ╚═══╝          


We see an initial block of gates that prepares the starting state on the first three qubits, followed by pairs of measurements and conditional bit flips, applied to pairs of qubits.



In [24]:
counts = create_half_filled_state(x)
print(f"sampled bit strings:\n{list(counts.keys())}")

sampled bit strings:
['000111', '001110', '010101', '011100', '100011', '101010', '110001', '111000']


Indeed, we created half-filled computational basis states, each with its own probability:

In [25]:
print("The probabilities for the bit strings are:")
for key, val in counts.items():
    print(f"    {key}: {val/shots*100:4.1f} %")

The probabilities for the bit strings are:
    000111: 24.0 %
    001110:  3.0 %
    010101:  5.0 %
    011100: 16.0 %
    100011: 15.0 %
    101010:  8.0 %
    110001:  3.0 %
    111000: 26.0 %


***Postselecting dynamically prepared states***

We can select only some of these half-filled states by postselecting on measurement outcomes:



In [29]:
@qml.qnode(dev)
def postselect_half_filled_state(x, selection):
  init_state(x)
  for w in range(3):
    #post select the measured qubit to match the selection criterion
    mcm=qml.measure(w, postselect = selection[w])
    qml.cond(~mcm, qml.X)(w+3)

  return qml.counts(wires=range(6))

As an example, suppose we wanted half-filled states that have a 0 in the first and a 1 in the third position. We do not postselect on the second qubit, which we can indicate by passing None to the postselect argument of measure(). Again, before running the circuit, let's draw it first:

In [30]:
selection = [0, None, 1]
print(qml.draw(postselect_half_filled_state)(x, selection))

0: ──RX(1.33)─╭●────╭X──┤↗₀├─────────────────────┤ ╭Counts
1: ──RX(1.63)─╰X─╭●─│────║───────┤↗├─────────────┤ ├Counts
2: ──RX(0.92)────╰X─╰●───║────────║──────┤↗₁├────┤ ├Counts
3: ──────────────────────║────X───║───────║──────┤ ├Counts
4: ──────────────────────║────║───║───X───║──────┤ ├Counts
5: ──────────────────────║────║───║───║───║────X─┤ ╰Counts
                         ╚════╝   ║   ║   ║    ║          
                                  ╚═══╝   ║    ║          
                                          ╚════╝          


Note the indicated postselection values next to the drawn MCMs.

Time to run the postselecting circuit:

In [31]:
counts = postselect_half_filled_state(x, selection)
postselected_shots = sum(counts.values())

print(f"Obtained {postselected_shots} out of {shots} samples after postselection.")
print("The probabilities for the postselected bit strings are:")
for key, val in counts.items():
    print(f"    {key}: {val/postselected_shots*100:4.1f} %")

Obtained 17 out of 100 samples after postselection.
The probabilities for the postselected bit strings are:
    001110: 35.3 %
    011100: 64.7 %


We successfully postselected on the desired properties of the computational basis state. Note that the number of returned samples is reduced, because those samples that do not meet the postselection criteria are discarded entirely.



***Dynamically correct quantum states***

If we do not want to postselect the prepared states but still would like to guarantee some of the qubits to be in a selected state, we can instead flip the corresponding pairs of bits if we measured the undesired state:



In [32]:
@qml.qnode(dev)
def create_selected_half_filled_state(x, selection):
  init_state(x)
  all_mcms =[]
  for w in range(3):
    #don't postselect on the selection criterion, but store the MCM for later
    mcm = qml.measure(w)
    qml.cond(~mcm, qml.X)(w+3)
    all_mcms.append(mcm)

  for w, sel, mcm in zip(range(3), selection, all_mcms):
    # If the postselection criterion is not None, flip the corresponding pair
    # of qubits conditioned on the mcm not satisfying the selection criterion
    if sel is not None:
      qml.cond(mcm !=sel, qml.X)(w)
      qml.cond(mcm !=sel, qml.X)(w+3)

  return qml.counts(wires=range(6))


print(qml.draw(create_selected_half_filled_state)(x,selection))




0: ──RX(1.33)─╭●────╭X──┤↗├─────────────────────X──────────┤ ╭Counts
1: ──RX(1.63)─╰X─╭●─│────║──────┤↗├─────────────║──────────┤ ├Counts
2: ──RX(0.92)────╰X─╰●───║───────║──────┤↗├─────║─────X────┤ ├Counts
3: ──────────────────────║───X───║───────║──────║──X──║────┤ ├Counts
4: ──────────────────────║───║───║───X───║──────║──║──║────┤ ├Counts
5: ──────────────────────║───║───║───║───║───X──║──║──║──X─┤ ╰Counts
                         ╚═══╩═══║═══║═══║═══║══╩══╝  ║  ║          
                                 ╚═══╝   ║   ║        ║  ║          
                                         ╚═══╩════════╩══╝          


We can see how the measured values are fed not only into the original conditioned operation, but also into the two additional bit flips for our “correction” procedure, as long as the selection criterion is not None. Let’s execute the circuit:



In [33]:
counts = create_selected_half_filled_state(x, selection)
selected_shots = sum(counts.values())

print(f"Obtained all {selected_shots} of {shots} samples because we did not postselect")
print("The probabilities for the selected bit strings are:")
for key, val in counts.items():
    print(f"    {key}: {val/selected_shots*100:4.1f} %")

Obtained all 100 of 100 samples because we did not postselect
The probabilities for the selected bit strings are:
    001110: 51.0 %
    011100: 49.0 %


Note that we kept all samples because we did not postselect, and that (in this particular case) this evened out the probabilities of measuring either of the two possible bit strings. Also, note that we conditionally applied the bit flip operators qml.X by comparing an MCM result to the corresponding selection criterion (mcm!=sel). More generally, MCM results in PennyLane can be processed with standard arithmetic operations. For details, see the introduction to MCMs and the documentation of measure().

And this is how to create dynamic circuits in PennyLane with mid-circuit measurements!